<a href="https://colab.research.google.com/github/Storm00212/QUANT/blob/main/Quant_volatility_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D


In [3]:
#Generate random trade returns
np.random.seed(42)

T = 1200            # time points
mu = 0.0003         # drift
sigma = 0.012       # volatility

returns = mu + sigma * np.random.randn(T)


In [4]:
# rolling statistics
def rolling_mean(x, w):
    return np.convolve(x, np.ones(w)/w, mode='valid')

def rolling_variance(x, w):
    return np.convolve(x**2, np.ones(w)/w, mode='valid')


In [5]:
# lag kernel volatility term
def lag_kernel(returns, t, lag, W):
    r_t = returns[t-W:t]
    r_l = returns[t-W-lag:t-lag]

    mu_t = np.mean(r_t)
    mu_l = np.mean(r_l)

    return np.mean((r_t - mu_t) * (r_l - mu_l))


In [6]:
# quantile  volatility floor
def volatility_floor(returns, t, W, K, q=0.75):
    variances = []
    for k in range(K):
        window = returns[t-k-W:t-k]
        variances.append(np.mean(window**2))
    return np.quantile(variances, q)
